In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.applications import ResNet50, ResNet152V2, DenseNet121
from keras.layers import Dense, Dropout, Activation, Input, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D,BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from keras import regularizers

In [3]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from mlxtend.plotting import plot_confusion_matrix

In [4]:
!pip install imutils

ERROR: Could not find a version that satisfies the requirement imutils (from versions: none)
ERROR: No matching distribution found for imutils


In [5]:
from imutils import paths
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import os
from tensorflow.keras.preprocessing import image

ModuleNotFoundError: No module named 'imutils'

## Creating the Data Directory and Dataset

In [ ]:
train_path = "../input/pediatric-pneumonia-chest-xray/Pediatric Chest X-ray Pneumonia/train"
test_path = "../input/pediatric-pneumonia-chest-xray/Pediatric Chest X-ray Pneumonia/test"

In [ ]:
normal_images = [img for img in os.listdir(f'{train_path}/NORMAL')]
pneumonia_images = [img for img in os.listdir(f'{train_path}/PNEUMONIA')]

In [ ]:
#Randomly select few images from the set to view (say, 5 images)
normal_select = np.random.choice(normal_images, 5, replace=False)
pneumonia_select = np.random.choice(pneumonia_images, 5, replace=False)

In [ ]:
#Plot the selected images
from tensorflow.keras.preprocessing import image

fig = plt.figure(figsize=(8,10))
for i in range(10):
    if i<5:
        fpath = f'{train_path}/NORMAL/{normal_select[i]}'
        label = 'NORMAL'
    else:
        fpath = f'{train_path}/PNEUMONIA/{pneumonia_select[i-5]}'
        label = 'PNEUMONIA'
    ax = fig.add_subplot(2,5,i+1)
    
    im = image.load_img(fpath)
    plt.imshow(im)
    plt.title(label)
    plt.axis('off')
plt.show()

In [ ]:
# #Generate an average image for each class from the the above classes of images
# #This will help understand the general characteristics of each class

# def img_to_matrix(fpath, flist, size=(64,64)):
#     for fname in flist:
#         full_path = fpath+fname
#         current_image = image.load_img(full_path, target_size = size, color_mode = 'grayscale')
        
#         #convert image to 64 x 64 matrix
#         img_mat1 = image.img_to_array(current_image)
#         img_mat1 = [img_mat1.ravel()]
#         try:
#             # concatenate different images
#             full_mat = np.concatenate((full_mat, img_mat1))
#         except UnboundLocalError: 
#             # if not assigned yet, assign one
#             full_mat = img_mat1
#     return full_mat

# normal_images1 = img_to_matrix(f'{train_path}/NORMAL/', normal_images)
# pneumonia_images1 = img_to_matrix(f'{train_path}/PNEUMONIA/', pneumonia_images)
        

In [ ]:
im = image.load_img("/kaggle/input/pediatric-pneumonia-chest-xray/Pediatric Chest X-ray Pneumonia/test/NORMAL/IM-0036-0001.jpeg")
plt.imshow(im)

In [ ]:
im.size

In [ ]:
# Fixed for our Pneumonia or not classes
NUM_CLASSES = 2
CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3
LEARNING_RATE = 1e-3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 53
STEPS_PER_EPOCH_VALIDATION = 11

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 8
BATCH_SIZE_VALIDATION = 8

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

test_datagen = ImageDataGenerator(rescale=1 / 255.0)

train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(IMAGE_RESIZE,IMAGE_RESIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE_TRAINING,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=69
)
valid_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(IMAGE_RESIZE,IMAGE_RESIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE_VALIDATION,
    class_mode="categorical",
    subset='validation',
    shuffle=True,
    seed=69
)
test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(IMAGE_RESIZE,IMAGE_RESIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE_TESTING,
    class_mode=None,
    shuffle=False,
    seed=69
)

In [ ]:
baseModel = DenseNet121(weights="imagenet", include_top=False, input_tensor=Input(shape=(IMAGE_RESIZE, IMAGE_RESIZE, 3)))
# construct the head of the model that will be placed on top of the
# the base model
model = Sequential()
model.add(baseModel)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))
for layer in baseModel.layers[:]:
    layer.trainable = False
print('conv_base is now NOT trainable')

In [ ]:
model.summary()

In [ ]:
# compile our model
print("Compiling the model...")
opt = Adam(lr=LEARNING_RATE, weight_decay=0.9)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy","AUC","Precision","Recall"])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

#cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = '../working/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [ ]:
# train the head of the network
print("training the head...")

history = model.fit(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
        validation_data=valid_generator,
        validation_steps=STEPS_PER_EPOCH_VALIDATION,
        callbacks=[cb_checkpointer]
)

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history)

In [ ]:
print(history.history.keys())

In [ ]:
model.load_weights("../working/best.hdf5")

In [ ]:
plt.figure(1, figsize = (15,8)) 
    
plt.subplot(221)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
    
plt.subplot(222)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 

plt.show()

In [ ]:
# Reset before each call to predict
test_generator.reset()

pred = model.predict(test_generator, steps = len(test_generator), verbose = 2)

predicted_class_indices = np.argmax(pred, axis = 1)

In [ ]:
TEST_DIR = '/kaggle/input/pediatric-pneumonia-chest-xray/Pediatric Chest X-ray Pneumonia/test/'
l = random.sample(range(0, len(test_generator.filenames)), 25)
print(l)

In [ ]:
f, ax = plt.subplots(5, 5, figsize = (15, 15))
for i in range(0,25):
    img = cv2.imread(TEST_DIR + test_generator.filenames[l[i]])
    
    
    # a if condition else b
    predicted_class = "Pneumonia" if predicted_class_indices[l[i]] else "Normal"

    ax[i//5, i%5].imshow(img)
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_title("Predicted:{}".format(predicted_class))    

plt.show()

In [ ]:
for i in range(0,25):
    print(test_generator.filenames[l[i]], predicted_class_indices[l[i]])

In [ ]:
def Targetization(folder_path, target_path): 
    label = []
    for path, tag in target_path:
        for filename in os.listdir(folder_path + path):
            img = cv2.imread(os.path.join(folder_path, path, filename)) # second param: 0 is b&w picture
            if img is not None:
                label.append(tag)
    return np.array(label)

base_dir= "../input/pediatric-pneumonia-chest-xray/Pediatric Chest X-ray Pneumonia/"
y_test = Targetization(base_dir, target_path=[("test/NORMAL/", 0,), ("test/PNEUMONIA/", 1,)])

In [ ]:
print(plot_confusion_matrix(confusion_matrix(y_test, predicted_class_indices), figsize=(5,5)))
print(accuracy_score(y_test, predicted_class_indices))
print(classification_report(y_test, predicted_class_indices))
plt.show()